In [77]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.mixture import GaussianMixture
import openpyxl
from dataprep.eda import plot, plot_correlation, create_report, plot_missing
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [78]:
# data = pd.read_excel('marketing_campaign.xlsx', engine = "openpyxl")
data = pd.read_csv('123.csv')


In [79]:
data.head(10)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.000,0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.000,1,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.000,0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.000,1,0,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.000,1,0,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,3,11,0
5,7446,1967,Master,Together,62513.000,0,1,09-09-2013,16,520,42,98,0,42,14,2,6,4,10,6,0,0,0,0,0,0,3,11,0
6,965,1971,Graduation,Divorced,55635.000,0,1,13-11-2012,34,235,65,164,50,49,27,4,7,3,7,6,0,0,0,0,0,0,3,11,0
7,6177,1985,PhD,Married,33454.000,1,0,08-05-2013,32,76,10,56,3,1,23,2,4,0,4,8,0,0,0,0,0,0,3,11,0
8,4855,1974,PhD,Together,30351.000,1,0,06-06-2013,19,14,0,24,3,3,2,1,3,0,2,9,0,0,0,0,0,0,3,11,1
9,5899,1950,PhD,Together,5648.000,1,1,13-03-2014,68,28,0,6,1,1,13,1,1,0,0,20,1,0,0,0,0,0,3,11,0


In [80]:
# plot(data)

In [81]:
data['Age'] = 2014 - data['Year_Birth']
data['Spending'] = data['MntWines'] + data['MntFruits'] + data[
    'MntMeatProducts'] + data['MntFishProducts'] + data[
        'MntSweetProducts'] + data['MntGoldProds']

last_date = date(2014, 10, 4)
data['Seniority'] = pd.to_datetime(data['Dt_Customer'], dayfirst = True, format = '%d-%m-%Y')
data['Seniority'] = pd.to_numeric(data['Seniority'].dt.date.apply(lambda x : (last_date - x)).dt.days, downcast = 'integer') / 30
data = data.rename(
    columns={
        'NumWebPurchases': "Web",
        'NumCatalogPurchases': 'Catalog',
        'NumStorePurchases': 'Store'
    })

data['Marital_Status'] = data['Marital_Status'].replace({
    'Divorced': 'Alone',
    'Single': 'Alone',
    'Married': 'In couple',
    'Together': 'In couple',
    'Absurd': 'Alone',
    'Widow': 'Alone',
    'YOLO': 'Alone'
})
data['Education'] = data['Education'].replace({
    'Basic': 'Undergraduate',
    '2n Cycle': 'Undergraduate',
    'Graduation': 'Postgraduate',
    'Master': 'Postgraduate',
    'PhD': 'Postgraduate'
})

data['Children'] = data['Kidhome'] + data['Teenhome']
data['Has_child'] = np.where(data.Children > 0, 'Has child', 'No child')
data['Children'].replace(
    {
        3: "3 children",
        2: '2 children',
        1: '1 child',
        0: "No child"
    },
    inplace=True)

# raname
# wine -> life insurance
# fruits -> car
# meat -> laptop
data = data.rename(
    columns={
        'MntWines': "LifeInsurance",
        'MntFruits': 'Cars',
        'MntMeatProducts': 'Laptop'
    })

data = data[[
    'Age', 'Education', 'Marital_Status', 'Income', 'Spending', 'Seniority',
    'Has_child', 'Children', 'LifeInsurance', 'Cars', 'Laptop'
]]

data.head()


,Age,Education,Marital_Status,Income,Spending,Seniority,Has_child,Children,LifeInsurance,Cars,Laptop
0,57,Postgraduate,Alone,58138.000,1617,25.333,No child,No child,635,88,546
1,60,Postgraduate,Alone,46344.000,27,7.000,Has child,2 children,11,1,6
2,49,Postgraduate,In couple,71613.000,776,13.633,No child,No child,426,49,127
3,30,Postgraduate,In couple,26646.000,53,7.867,Has child,1 child,11,4,20
4,33,Postgraduate,In couple,58293.000,422,8.600,Has child,1 child,173,43,118


In [82]:
data = data.dropna(subset=['Income'])
data = data[data['Income'] < 600000]

In [83]:
scaler = StandardScaler()
dataset_temp = data[['Income', 'Seniority', 'Spending']]
X_std = scaler.fit_transform(dataset_temp)
X = normalize(X_std, norm='l2')

In [84]:
gmm = GaussianMixture(n_components=4,
                      covariance_type='spherical',
                      max_iter=2000,
                      random_state=5).fit(X)
labels = gmm.predict(X)
dataset_temp['Cluster'] = labels
dataset_temp = dataset_temp.replace({
    0: 'Stars',
    1: 'Need attention',
    2: 'High potential',
    3: 'Leaky bucket'
})
data = data.merge(dataset_temp.Cluster, left_index=True, right_index=True)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [85]:
pd.options.display.float_format = "{:.0f}".format
summary = data[['Income', 'Spending', 'Seniority', 'Cluster']]
summary.set_index("Cluster", inplace=True)
summary = summary.groupby('Cluster').describe().transpose()
summary.head()

Cluster       High potential  Leaky bucket  Need attention  Stars
Income count             462           585             641    527
       mean            73438         34790           37705  69571
       std             13753         12090           12397  11998
       min             49090          2447            1730  44802
       25%             65298         26490           28839  60895

In [86]:
#Create Age segment
cut_labels_Age = ['Young', 'Adult', 'Mature', 'Senior']
cut_bins = [0, 30, 45, 65, 120]
data['Age_group'] = pd.cut(data['Age'], bins=cut_bins, labels=cut_labels_Age)
#Create Income segment
cut_labels_Income = [
    'Low income', 'Low to medium income', 'Medium to high income',
    'High income'
]
data['Income_group'] = pd.qcut(data['Income'], q=4, labels=cut_labels_Income)
#Create Seniority segment
cut_labels_Seniority = [
    'New customers', 'Discovering customers', 'Experienced customers',
    'Old customers'
]
data['Seniority_group'] = pd.qcut(data['Seniority'],
                                  q=4,
                                  labels=cut_labels_Seniority)
data = data.drop(columns=['Age', 'Income', 'Seniority'])


In [87]:
cut_labels = ['Low consumer', 'Frequent consumer', 'Biggest consumer']
data['LifeInsurance_segment'] = pd.qcut(data['LifeInsurance'][data['LifeInsurance'] > 0],
                                q=[0, .25, .75, 1],
                                labels=cut_labels).astype("object")
data['Car_segment'] = pd.qcut(data['Cars'][data['Cars'] > 0],
                                 q=[0, .25, .75, 1],
                                 labels=cut_labels).astype("object")
data['Laptop_segment'] = pd.qcut(data['Laptop'][data['Laptop'] > 0],
                               q=[0, .25, .75, 1],
                               labels=cut_labels).astype("object")
# data['Fish_segment'] = pd.qcut(data['Fish'][data['Fish'] > 0],
#                                q=[0, .25, .75, 1],
#                                labels=cut_labels).astype("object")
# data['Sweets_segment'] = pd.qcut(data['Sweets'][data['Sweets'] > 0],
#                                  q=[0, .25, .75, 1],
#                                  labels=cut_labels).astype("object")
# data['Gold_segment'] = pd.qcut(data['Gold'][data['Gold'] > 0],
#                                q=[0, .25, .75, 1],
#                                labels=cut_labels).astype("object")
data.replace(np.nan, "Non consumer", inplace=True)
data.drop(
    columns=['Spending', 'LifeInsurance', 'Cars', 'Laptop'],
    inplace=True)
data = data.astype(object)

In [89]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 999)
pd.options.display.float_format = "{:.3f}".format
association = data.copy()
df = pd.get_dummies(association)
min_support = 0.08
max_len = 10
frequent_items = apriori(df,
                         use_colnames=True,
                         min_support=min_support,
                         max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)

product = 'Laptop'
segment = 'Biggest consumer'
target = '{\'%s_segment_%s\'}' % (product, segment)
results_personnal_care = rules[rules['consequents'].astype(str).str.contains(
    target, na=False)].sort_values(by='confidence', ascending=False)
results_personnal_care.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
7259,"(Children_No child, Income_group_High income, Cluster_High potential)",(Laptop_segment_Biggest consumer),0.092,0.250,0.081,0.887,3.545,0.058,6.619
7132,"(Has_child_No child, Income_group_High income, Cluster_High potential)",(Laptop_segment_Biggest consumer),0.092,0.250,0.081,0.887,3.545,0.058,6.619
9931,"(Has_child_No child, Children_No child, Income_group_High income, Cluster_High potential)",(Laptop_segment_Biggest consumer),0.092,0.250,0.081,0.887,3.545,0.058,6.619
7145,"(LifeInsurance_segment_Biggest consumer, Has_child_No child, Income_group_High income)",(Laptop_segment_Biggest consumer),0.104,0.250,0.091,0.878,3.511,0.065,6.160
9961,"(LifeInsurance_segment_Biggest consumer, Has_child_No child, Children_No child, Income_group_High income)",(Laptop_segment_Biggest consumer),0.104,0.250,0.091,0.878,3.511,0.065,6.160
